In [4]:
from torch.distributions import Categorical
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
# class Pi(nn.Module):
#     def __init__(self, in_dim, out_dim):
#         super(Pi, self).__init__()
#         layers = [
#             nn.Linear(in_dim, 64),
#             nn.ReLU(),
#             nn.Linear(64, out_dim)
#         ]
#         self.model = nn.Sequential(*layers)
#         self.onpolicy_reset()
#         self.train() # set training mode
    
#     def onpolicy_reset(self):
#         self.log_probs = []
#         self.rewards = []
    
#     def forward(self, x):
#         pdparam = self.model(x)
#         return pdparam
    
#     def act(self, state):
#         x = torch.from_numpy(state.astype(np.float32))
#         pdparam = self. forward(x) # forward pass
#         pd = Categorical(logits=pdparam) # probability distributions
#         action = pd.sample() 
#         log_prob = pd.log_prob(action)
#         self.log_probs.append(log_prob)
#         return action.item()

In [8]:
# def train(pi, optimizer):
#     # Inner gradient-ascent loop of REINFORCE algorithm
#     T = len(pi.rewards)
#     rets = np.empty(T, dtype=np.float32)
#     future_ret = 0.0
#     # compute the returns efiiciently
#     for t in reversed(range(T)):
#         future_ret = pi.rewards[t] + gamma * future_ret
#         rets[t] = future_ret
#     rets = torch.tensor(rets)
#     log_probs = torch.stack(pi.log_probs)
#     loss = - log_probs * rets 
#     loss = torch.sum(loss)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     return loss

In [9]:
# env = gym.make('CartPole-v0')
# in_dim = env.observation_space.shape[0]
# out_dim = env.action_space.n
# pi = Pi(in_dim,out_dim)
# optimizer = optim.Adam(pi.parameters(), lr=0.01)
# for epi in range(300):
#     state = env.reset()
#     for t in range(200):
#         action = pi.act(state)
#         state, reward, done, _ = env.step(action)
#         pi.rewards.append(reward)
#         env.render()
#         if done:
#             break
#     loss = train(pi, optimizer)
#     total_reward = sum(pi.rewards)
#     solved = total_reward > 195.0
#     pi.onpolicy_reset()
#     print(f'Episode {epi}, loss: {loss}, \
# 77          total_reward: {total_reward}, solved: {solved}')

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    

In [11]:
class Policy(nn.Module):
    def __init__(self, s_size=4, h_size=16, a_size=2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)